In [1]:
import sys
import time
import math
import itertools

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from scipy.special import logit
from scipy.stats import norm

import tensorflow as tf
from keras import layers, models, datasets

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import StepLR, LambdaLR
import torch.autograd.profiler as profiler

from sklearn.model_selection import train_test_split, LeaveOneOut, StratifiedKFold, cross_val_predict
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, PowerTransformer
from sklearn.metrics import f1_score, log_loss, accuracy_score
from sklearn.linear_model import LogisticRegression

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [2]:
def calculate_metrics(model, data_tensor, labels_tensor, batch_size=1024, num_features=22):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for start_idx in range(0, len(data_tensor), batch_size):
            end_idx = min(start_idx + batch_size, len(data_tensor))
            inputs = data_tensor[start_idx:end_idx].view(-1, num_features)
            labels = labels_tensor[start_idx:end_idx]

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    return accuracy, f1

In [3]:
class CustomDataLoader:
    def __init__(self, features, labels, validation_size=0.2, random_state=42, classification=True):        
        if validation_size > 0.0:
            stratify = labels if classification else None
            train_data, val_data, train_labels, val_labels = train_test_split(
                features, labels, test_size=validation_size, stratify=stratify, random_state=random_state
            )
            
            self.val_data_tensor = torch.tensor(val_data).float().to(device)
            
            if classification:
                self.val_labels_tensor = torch.tensor(val_labels).long().to(device)

            else:
                self.val_labels_tensor =torch.tensor(val_labels).float().to(device)
        else:
            train_data, train_labels = features, labels
            self.val_data_tensor, self.val_labels_tensor = None, None
        
        self.train_data_tensor = torch.tensor(train_data).float().to(device)

        if classification:
            self.train_labels_tensor = torch.tensor(train_labels).long().to(device)
        else:
            self.train_labels_tensor = torch.tensor(train_labels).float().to(device)

        torch.manual_seed(random_state)
        indices = torch.randperm(len(self.train_data_tensor))

        self.train_data_tensor = self.train_data_tensor[indices]
        self.train_labels_tensor = self.train_labels_tensor[indices]

In [4]:
def evaluate_model(model, custom_train_loader, criterion, optimizer, num_epochs, scheduler, batch_size=1024, num_features=22, early_stopping_patience=10):
    best_val_loss = float('inf')
    best_epoch = 0
    patience_counter = 0
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        model.train()
        i = 0
        total_loss = 0
        num_items = 0

        for start_idx in range(0, len(custom_train_loader.train_data_tensor), batch_size):
            end_idx = min(start_idx + batch_size, len(custom_train_loader.train_data_tensor))
            inputs = custom_train_loader.train_data_tensor[start_idx:end_idx].view(-1, num_features)
            labels = custom_train_loader.train_labels_tensor[start_idx:end_idx]

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels, model)
            loss.backward()
            optimizer.step()
            scheduler.step()
            running_loss += loss.item() * len(labels)
            total_loss += loss.item() * len(labels)
            num_items += len(labels)

            i += 1

        if epoch % 10 == 0:
            model.eval()
            for param_group in optimizer.param_groups:
                print("Learning Rate:", param_group['lr'])

            train_reg_loss = 0.0
            val_loss = 0.0
            with torch.no_grad():
                for start_idx in range(0, len(custom_train_loader.train_data_tensor), batch_size):
                    end_idx = min(start_idx + batch_size, len(custom_train_loader.train_data_tensor))
                    inputs = custom_train_loader.train_data_tensor[start_idx:end_idx].view(-1, num_features)
                    labels = custom_train_loader.train_labels_tensor[start_idx:end_idx]
        
                    outputs = model(inputs)
                    train_reg_loss += criterion.regular_loss(outputs, labels).item() * len(labels)

                for start_idx in range(0, len(custom_train_loader.val_data_tensor), batch_size):
                    end_idx = min(start_idx + batch_size, len(custom_train_loader.val_data_tensor))
                    val_inputs = custom_train_loader.val_data_tensor[start_idx:end_idx].view(-1, num_features)
                    val_labels = custom_train_loader.val_labels_tensor[start_idx:end_idx]
    
                    val_outputs = model(val_inputs)
                    val_loss += criterion.regular_loss(val_outputs, val_labels).item() * len(val_labels)
    
            avg_train_loss = running_loss / len(custom_train_loader.train_data_tensor)
            avg_val_loss = val_loss / len(custom_train_loader.val_data_tensor)
    
            train_accuracy, train_f1 = calculate_metrics(model, custom_train_loader.train_data_tensor, custom_train_loader.train_labels_tensor, batch_size, num_features)
            val_accuracy, val_f1 = calculate_metrics(model, custom_train_loader.val_data_tensor, custom_train_loader.val_labels_tensor, batch_size, num_features)
    
            print(f'Epoch {epoch + 1}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}')
            print(f'Epoch {epoch + 1}, Training Loss: {train_reg_loss / len(custom_train_loader.train_data_tensor)}, Validation Loss: {avg_val_loss}')
            print(f'Training Accuracy: {train_accuracy}, Training F1 Score: {train_f1}')
            print(f'Validation Accuracy: {val_accuracy}, Validation F1 Score: {val_f1}')
            print()
            
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                best_epoch = epoch + 1
                patience_counter = 0
            else:
                patience_counter += 10
                if patience_counter >= early_stopping_patience:
                    print(f'Early stopping triggered after {epoch + 1} epochs.')
                    print(f'Best Validation Loss: {best_val_loss} from Epoch {best_epoch}')
                    break

    if patience_counter < early_stopping_patience:
        print(f'Best Validation Loss after {num_epochs} epochs: {best_val_loss} from Epoch {best_epoch}')

In [35]:
data_dl = pd.read_csv('/kaggle/input/playground-series-s4e10/train.csv')
data_og = pd.read_csv('/kaggle/input/loan-approval-prediction/credit_risk_dataset.csv')

data_dl = data_dl.drop(["id"], axis=1)

median_emp_length = data_og['person_emp_length'].median()
median_int_rate = data_og['loan_int_rate'].median()

data_dl['source'] = 0
data_og['source'] = 1

data = pd.concat([data_dl, data_og], ignore_index=True)

In [40]:
data = data_dl

data['person_emp_length_missing'] = data['person_emp_length'].isna().astype(int)
data['loan_int_rate_missing'] = data['loan_int_rate'].isna().astype(int)

data['person_emp_length'] = data['person_emp_length'].fillna(median_emp_length)
data['loan_int_rate'] = data['loan_int_rate'].fillna(median_int_rate)

# grade_mapping = {'A': 7, 'B': 6, 'C': 5, 'D': 4, 'E': 3, 'F': 2, 'G': 1}
# data['loan_grade'] = data['loan_grade'].map(grade_mapping)

# purpose_mapping = {
#     'DEBTCONSOLIDATION': 1,
#     'HOMEIMPROVEMENT': 2,
#     'MEDICAL': 3,
#     'PERSONAL': 4,
#     'EDUCATION': 5,
#     'VENTURE': 6
# }
# data['loan_intent'] = data['loan_intent'].map(purpose_mapping)

# home_ownership_mapping = {
#     'OWN': 1,
#     'MORTGAGE': 2,
#     'OTHER': 3,
#     'RENT': 4
# }
# data['person_home_ownership'] = data['person_home_ownership'].map(home_ownership_mapping)

X = data.drop(["loan_status"], axis=1)
X = pd.get_dummies(X, drop_first=True)
y = data["loan_status"]

column_to_log = [
    'person_age',
    'person_income',
]

column_to_sqrt = [
    'person_emp_length',
    'loan_percent_income',
]

for col in column_to_log:
    if (X[col] <= 0).any():
        print(f"Column '{col}' contains non-positive values. Adding 1 to avoid log of non-positive numbers.")
        X[col] = np.log(X[col] + 1)
    else:
        X[col] = np.log(X[col])

for col in column_to_sqrt:
    if (X[col] < 0).any():
        print(f"Column '{col}' contains negative values. Setting negative values to NaN before applying sqrt.")
        X[col] = np.sqrt(X[col].clip(lower=0))
    else:
        X[col] = np.sqrt(X[col])

print(data.isnull().sum())
print(X.columns)
print(X.shape, y.shape)
print(X.columns.get_loc('source'))

person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
loan_status                   0
source                        0
person_emp_length_missing     0
loan_int_rate_missing         0
dtype: int64
Index(['person_age', 'person_income', 'person_emp_length', 'loan_amnt',
       'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length',
       'source', 'person_emp_length_missing', 'loan_int_rate_missing',
       'person_home_ownership_OTHER', 'person_home_ownership_OWN',
       'person_home_ownership_RENT', 'loan_intent_EDUCATION',
       'loan_intent_HOMEIMPROVEMENT', 'loan_intent_MEDICAL',
       'loan_intent_PERSONAL', 'loan_intent_VENTURE', 'loan_grade_B',
       'loan_grade_C', 'loa

In [38]:
x_scaler = StandardScaler()
x_scaled = x_scaler.fit_transform(X)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print(x_scaled.shape)

(32581, 25)


In [41]:
x_scaled = x_scaler.transform(X)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print(x_scaled)
print(x_scaled.shape)

[[ 1.56429348 -0.81624065 -1.86421602 ... -0.08632538 -0.04436441
  -0.46268575]
 [-1.06299707  0.01394005  0.50889981 ... -0.08632538 -0.04436441
  -0.46268575]
 [ 0.33310246 -1.16062639  0.8760221  ... -0.08632538 -0.04436441
  -0.46268575]
 ...
 [-0.83835155 -0.41203131  0.69904203 ... -0.08632538 -0.04436441
  -0.46268575]
 [-1.06299707 -1.08852135 -0.49409696 ... -0.08632538 -0.04436441
  -0.46268575]
 [ 0.67014014  0.52994887 -0.49409696 ... -0.08632538 -0.04436441
  -0.46268575]]
(58645, 25)


In [42]:
feature_means = x_scaled.mean(axis=0)
feature_variances = x_scaled.var(axis=0)
feature_mins = x_scaled.min(axis=0)
feature_maxs = x_scaled.max(axis=0)

feature_stats_scaled_full = pd.DataFrame({
    'Mean': feature_means,
    'Variance': feature_variances,
    'Min': feature_mins,
    'Max': feature_maxs
})

print("Mean, Variance, Min, and Max of Scaled Features:")
print(feature_stats_scaled_full)

Mean, Variance, Min, and Max of Scaled Features:
        Mean  Variance       Min        Max
0  -0.027383  0.948764 -1.544665   7.635124
1   0.056332  0.651183 -4.561322   6.238936
2  -0.014743  0.991884 -1.864216   8.880523
3  -0.058813  0.774514 -1.437739   4.019404
4  -0.107655  0.969797 -1.813891   3.962397
5  -0.077807  0.797797 -3.079228   4.070171
6   0.002305  0.987326 -0.938167   5.966992
7  -1.000000  0.000000 -1.000000  -1.000000
8  -0.168065  0.000000 -0.168065  -0.168065
9  -0.325196  0.000000 -0.325196  -0.325196
10 -0.030876  0.462923 -0.057402  17.421117
11 -0.095483  0.693581 -0.293499   3.407161
12  0.033819  0.998211 -1.009591   0.990500
13  0.028057  1.041726 -0.496967   2.012205
14 -0.011356  0.971681 -0.352723   2.835088
15  0.000278  1.000448 -0.478548   2.089656
16  0.003561  1.006262 -0.451695   2.213885
17 -0.012688  0.978196 -0.461414   2.167251
18  0.058028  1.041196 -0.687208   1.455163
19 -0.025161  0.961272 -0.497207   2.011234
20 -0.080934  0.793382 -0.3

In [43]:
class CustomLoss(nn.Module):
    def __init__(self, criterion, f1_lambda, f2_lambda, l1_lambda, l2_lambda, wa_lambda):
        super(CustomLoss, self).__init__()
        self.criterion = criterion
        self.f1_lambda = f1_lambda
        self.f2_lambda = f2_lambda
        self.l1_lambda = l1_lambda
        self.l2_lambda = l2_lambda
        self.wa_lambda = wa_lambda
        self.i = 0

    def forward(self, outputs, labels, model): 
        f1_loss = 0.0
        f2_loss = 0.0
        l1_loss = 0.0
        l2_loss = 0.0

        for name, module in model.named_modules():
            if isinstance(module, CustomActivation):
                f1_loss += (module.a ** 2).sum() + (module.b ** 2).sum()
                f2_loss += ((module.a - module.b) ** 2).sum()

            if isinstance(module, nn.Linear):
                l1_loss += torch.norm(module.weight, 1)
                l2_loss += torch.norm(module.weight, 2) ** 2

        total_loss = (self.criterion(outputs, labels)
                      + self.f1_lambda * f1_loss
                      + self.f2_lambda * f2_loss
                      + self.l1_lambda * l1_loss
                      + self.l2_lambda * l2_loss)
        self.i += 1

        return total_loss

    def compute_gradient_magnitude(self, model):
        total_abs_sum = 0.0
        for param in model.parameters():
            if param.grad is not None:
                total_abs_sum += param.grad.abs().sum().item()
        self.grad_magnitude = total_abs_sum

    def regular_loss(self, outputs, labels):
        return self.criterion(outputs, labels)

In [44]:
class CustomActivation1d(nn.Module):
    def __init__(self, num_features):
        super(CustomActivation1d, self).__init__()
        self.a = nn.Parameter(torch.zeros(num_features))
        self.b = nn.Parameter(torch.zeros(num_features))
        
        self.local_bias = nn.Parameter(torch.zeros(num_features))
        self.global_bias = nn.Parameter(torch.tensor(0.0))

    def forward(self, x):
        x = x + self.local_bias
        x = torch.where(x < 0, self.a * x, self.b * x)
        return x.sum(dim=-1) + self.global_bias

In [45]:
class CustomActivation(nn.Module):
    def __init__(self, num_features):
        super(CustomActivation, self).__init__()
        self.a = nn.Parameter(torch.zeros(num_features, num_features))
        self.b = nn.Parameter(torch.zeros(num_features, num_features))
        
        self.local_bias = nn.Parameter(torch.zeros(num_features, num_features))
        self.global_bias = nn.Parameter(torch.zeros(num_features))

        with torch.no_grad():
            self.a.fill_diagonal_(1)
            self.b.fill_diagonal_(1)
            
    def forward(self, x):
        batch_size, num_features = x.shape
        x = x.unsqueeze(-1).expand(-1, -1, num_features)
        x = x + self.local_bias
        x = torch.where(x < 0, self.a * x, self.b * x)
        return x.sum(dim=1) + self.global_bias

In [ ]:
class CustomLinear(nn.Module):
    def __init__(self, num_features, num_outputs, init_identity=False):
        super(CustomLinear, self).__init__()
        
        if init_identity and num_features != num_outputs:
            raise ValueError("For identity initialization, num_features must equal num_outputs.")

        self.linear = nn.Linear(num_features, num_outputs, bias=True)
        
        with torch.no_grad():
            self.linear.bias.zero_()

            if init_identity:
                self.linear.weight.copy_(torch.eye(num_features, num_outputs))
            else:
                self.linear.weight.zero_()

    def forward(self, x):
        return self.linear(x)

In [ ]:
"""
class TabularDenseNet(nn.Module):
    def __init__(self, input_size, output_size, num_control_points, num_layers, window_size):
        super(TabularDenseNet, self).__init__()
        self.layers = nn.ModuleList()
        
        if num_layers % 2 == 1:
            self.layers.append(CustomLinear(input_size, input_size, init_identity=True))
            # self.layers.append(CustomActivation(input_size, window_size, num_control_points, init_identity=True))
            num_layers -= 1
            input_size *= 2
            
        for i in range(num_layers):
            if i % 2 == 0:
                self.layers.append(CustomLinear(input_size, input_size, init_identity=True))
            else:
                # self.layers.append(CustomActivation(input_size, window_size, num_control_points, init_identity=True))
                self.layers.append(CustomLinear(input_size, input_size, init_identity=True))

            input_size *= 2

        self.final = CustomLinear(input_size, output_size, init_identity=False)
        self.final_act = CustomActivation(output_size, window_size, num_control_points, init_identity=True)
        
    def forward(self, x):
        outputs = [x]

        for layer in self.layers:
            concatenated_outputs = torch.cat(outputs, dim=-1)
            outputs.append(F.relu(layer(concatenated_outputs)))

        concatenated_outputs = torch.cat(outputs, dim=-1)
        x = self.final(concatenated_outputs)
        x = self.final_act(x)
        return x
"""
print("")

In [46]:
class TabularDenseNet(nn.Module):
    def __init__(self, input_size, output_size, num_control_points, num_layers):
        super(TabularDenseNet, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(num_layers):
            self.layers.append(CustomActivation(input_size))
            input_size *= 2

        self.final_layer = CustomActivation1d(input_size)

    def forward(self, x):
        outputs = [x]

        for layer in self.layers:
            concatenated_outputs = torch.cat(outputs, dim=-1)
            outputs.append(layer(concatenated_outputs))

        final_out = self.final_layer(torch.cat(outputs, dim=-1))
        return torch.stack([final_out, -final_out], dim=-1)

    # def forward(self, x):
    #     outputs = [x]
    #     summed_total = torch.zeros(x.size(0), device=x.device)
    #     summed_total += self.bias

    #     for layer, layer_norm in zip(self.layers, self.layer_norms):
    #         concatenated_outputs = torch.cat(outputs, dim=-1)
    #         inter_features = layer_norm(layer(concatenated_outputs))
    #         summed_feature = inter_features.sum(dim=-1)

    #         outputs.append(inter_features)
    #         outputs.append(summed_feature.unsqueeze(-1))
    #         summed_total += summed_feature

    #     print(torch.cat(outputs, dim=-1).shape)
    #     return torch.stack([summed_total, -summed_total], dim=-1)

    # def forward(self, x):
    #     batch_size = x.size(0)
    #     device = x.device
    #     L = self.num_layers
    #     D = self.input_dim

    #     total_features = 6656
    #     concatenated_features = torch.zeros(batch_size, total_features, device=device)

    #     concatenated_features[:, :D] = x

    #     current_pos = D
    #     summed_total = self.bias.expand(batch_size)

    #     for i, (layer, layer_norm) in enumerate(zip(self.layers, self.layer_norms)):
    #         current_features = concatenated_features[:, :current_pos]
    #         inter_features = (layer(current_features))
    #         summed_feature = inter_features.sum(dim=-1, keepdim=True)

    #         concatenated_features[:, current_pos:current_pos + current_pos] = inter_features
    #         concatenated_features[:, current_pos + current_pos + 1] = summed_feature.squeeze(-1)

    #         current_pos += current_pos + 1
    #         summed_total = summed_total + summed_feature.squeeze(-1)

    #     return torch.stack([summed_total, -summed_total], dim=-1)

In [47]:
custom_train_loader = CustomDataLoader(x_scaled, y_encoded, validation_size=0.1, random_state=0, classification=True)
print(custom_train_loader.train_data_tensor.shape)

torch.Size([52780, 25])


In [48]:
num_features = 25
num_classes = 2
num_control_points = 21
num_epochs = 10000
batch_size = 730 * 1

In [49]:
torch.cuda.empty_cache()

In [50]:
def custom_lr_lambda(step):
    num_step_threshold = 100

    if step < num_step_threshold:
        return step / num_step_threshold
    if step == num_step_threshold:
        print("here")
    return 0.99999 ** (step - num_step_threshold)

start_time = time.time()

optimizer = optim.Adam(model.parameters(), lr=0.001 * 1.0)
scheduler = LambdaLR(optimizer, lr_lambda=custom_lr_lambda)

criterion = CustomLoss(nn.CrossEntropyLoss(), 0.0, 0.0, 0.0, 0.0, 0.0)
evaluate_model(model, custom_train_loader, criterion, optimizer, 500, scheduler, batch_size, num_features, early_stopping_patience=10000)

print(f"Execution time: {(time.time() - start_time):.6f} seconds")

Learning Rate: 0.00073
Epoch 1, Training Loss: 0.1929988909700505, Validation Loss: 0.18885775679327033
Epoch 1, Training Loss: 0.1882823111348958, Validation Loss: 0.18885775679327033
Training Accuracy: 0.9486737400530504, Training F1 Score: 0.9457383289777652
Validation Accuracy: 0.947996589940324, Validation F1 Score: 0.9453974942513467

here
Learning Rate: 0.0009929946177428413
Epoch 11, Training Loss: 0.17267540987692534, Validation Loss: 0.1793293354386513
Epoch 11, Training Loss: 0.17210056624569736, Validation Loss: 0.1793293354386513
Training Accuracy: 0.9502463054187192, Training F1 Score: 0.9475322487788732
Validation Accuracy: 0.9493606138107417, Validation F1 Score: 0.9469281521237626

Learning Rate: 0.0009857721151293795
Epoch 21, Training Loss: 0.1690200964873286, Validation Loss: 0.17698571255203505
Epoch 21, Training Loss: 0.16873886605619526, Validation Loss: 0.17698571255203505
Training Accuracy: 0.9504736642667677, Training F1 Score: 0.9478049175727021
Validation Ac

KeyboardInterrupt: 

In [19]:
def custom_lr_lambda(step):
    num_step_threshold = 100

    if step < num_step_threshold:
        return step / num_step_threshold
    if step == num_step_threshold:
        print("here")
    return 0.99999 ** (step - num_step_threshold)

start_time = time.time()
model = TabularDenseNet(num_features, num_classes, 1, 1).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001 * 1.0)
scheduler = LambdaLR(optimizer, lr_lambda=custom_lr_lambda)

criterion = CustomLoss(nn.CrossEntropyLoss(), 0.0, 0.0, 0.0, 0.0, 0.0)
evaluate_model(model, custom_train_loader, criterion, optimizer, 200, scheduler, batch_size, num_features, early_stopping_patience=10000)

print(f"Execution time: {(time.time() - start_time):.6f} seconds")

Learning Rate: 0.00041
Epoch 1, Training Loss: 0.655168993688851, Validation Loss: 0.576732883079537
Epoch 1, Training Loss: 0.5773437749671949, Validation Loss: 0.576732883079537
Training Accuracy: 0.7837800968556033, Training F1 Score: 0.6907128531495372
Validation Accuracy: 0.783369131635471, Validation F1 Score: 0.6903129502633537

here
Learning Rate: 0.0009964961353604535
Epoch 11, Training Loss: 0.29465572077807134, Validation Loss: 0.30033097432851424
Epoch 11, Training Loss: 0.29160624468898183, Validation Loss: 0.30033097432851424
Training Accuracy: 0.8927085464838688, Training F1 Score: 0.887580517860501
Validation Accuracy: 0.8895366676894753, Validation F1 Score: 0.8841371518181538

Learning Rate: 0.0009924188449759277
Epoch 21, Training Loss: 0.26235879765884845, Validation Loss: 0.2718926128542983
Epoch 21, Training Loss: 0.2607936156150971, Validation Loss: 0.2718926128542983
Training Accuracy: 0.9114657936020736, Training F1 Score: 0.9070734432749952
Validation Accuracy

In [16]:
def custom_lr_lambda(step):
    num_step_threshold = 100

    if step < num_step_threshold:
        return step / num_step_threshold
    if step == num_step_threshold:
        print("here")
    return 0.99999 ** (step - num_step_threshold)

start_time = time.time()
model = TabularDenseNet(num_features, num_classes, 1, 1).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001 * 1.0)
scheduler = LambdaLR(optimizer, lr_lambda=custom_lr_lambda)

criterion = CustomLoss(nn.CrossEntropyLoss(), 0.0, 0.0, 0.0, 0.0, 0.0)
evaluate_model(model, custom_train_loader, criterion, optimizer, 10000, scheduler, batch_size, num_features, early_stopping_patience=10000)

print(f"Execution time: {(time.time() - start_time):.6f} seconds")

Learning Rate: 0.00041
Epoch 1, Training Loss: 0.655168993688851, Validation Loss: 0.576732883079537
Epoch 1, Training Loss: 0.5773437749671949, Validation Loss: 0.576732883079537
Training Accuracy: 0.7837800968556033, Training F1 Score: 0.6907128531495372
Validation Accuracy: 0.783369131635471, Validation F1 Score: 0.6903129502633537

here
Learning Rate: 0.0009964961353604535
Epoch 11, Training Loss: 0.29465572077807134, Validation Loss: 0.30033097432851424
Epoch 11, Training Loss: 0.29160624468898183, Validation Loss: 0.30033097432851424
Training Accuracy: 0.8927085464838688, Training F1 Score: 0.887580517860501
Validation Accuracy: 0.8895366676894753, Validation F1 Score: 0.8841371518181538

Learning Rate: 0.0009924188449759277
Epoch 21, Training Loss: 0.26235879765884845, Validation Loss: 0.2718926128542983
Epoch 21, Training Loss: 0.2607936156150971, Validation Loss: 0.2718926128542983
Training Accuracy: 0.9114657936020736, Training F1 Score: 0.9070734432749952
Validation Accuracy

KeyboardInterrupt: 

In [135]:
def custom_lr_lambda(step):
    num_step_threshold = 100

    if step < num_step_threshold:
        return step / num_step_threshold
    if step == num_step_threshold:
        print("here")
    return 0.99999 ** (step - num_step_threshold)

start_time = time.time()
model = TabularDenseNet(num_features, num_classes, 1, 1).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001 * 1.0)
scheduler = LambdaLR(optimizer, lr_lambda=custom_lr_lambda)

criterion = CustomLoss(nn.CrossEntropyLoss(), 0.0, 0.0, 0.0, 0.0, 0.0)
evaluate_model(model, custom_train_loader, criterion, optimizer, 10000, scheduler, batch_size, num_features, early_stopping_patience=10000)

print(f"Execution time: {(time.time() - start_time):.6f} seconds")

here
Learning Rate: 0.0009998700077997146
Epoch 1, Training Loss: 0.49528662510020316, Validation Loss: 0.3185676628562242
Epoch 1, Training Loss: 0.3205098198033491, Validation Loss: 0.3185676628562242
Training Accuracy: 0.8678732811224925, Training F1 Score: 0.8451023475456597
Validation Accuracy: 0.8712046475939932, Validation F1 Score: 0.8501742789721651

Learning Rate: 0.0009886350177801951
Epoch 11, Training Loss: 0.21353975831286232, Validation Loss: 0.22155042180500964
Epoch 11, Training Loss: 0.2119047555450359, Validation Loss: 0.22155042180500964
Training Accuracy: 0.9322436451773991, Training F1 Score: 0.928668983024037
Validation Accuracy: 0.9266688589279842, Validation F1 Score: 0.9228546861033203

Learning Rate: 0.0009775262691718131
Epoch 21, Training Loss: 0.2003762004482626, Validation Loss: 0.21076466172368435
Epoch 21, Training Loss: 0.19941092322888726, Validation Loss: 0.21076466172368435
Training Accuracy: 0.937079034870833, Training F1 Score: 0.9336873602605845


KeyboardInterrupt: 

In [136]:
for name, param in model.named_parameters():
    print(name)
    print(param)

layers.0.a
Parameter containing:
tensor([[ 4.4564e+00,  4.3177e-01,  5.1845e-02,  1.4698e-01, -1.4075e+00,
          1.4098e-01,  5.7946e-01, -1.2160e-01, -7.6425e-01,  9.1887e-02,
         -1.4741e-01, -4.8555e-01,  1.1405e-01, -5.2058e-01,  2.6454e+00,
         -8.5377e-01,  6.9552e-02, -2.0482e-01, -5.8601e-02,  2.7055e-01,
         -9.0596e-01, -1.8602e+00,  1.9821e-01, -7.3506e-02,  2.9277e-02],
        [ 7.1102e-02,  7.7473e-01, -3.9996e-02, -1.4458e-01, -5.5886e-01,
         -2.6232e-01, -5.2604e-01,  6.6752e-01, -4.2569e-02, -2.9622e-01,
         -6.3417e-01,  6.3476e-01, -2.3447e+00, -4.2899e-01, -2.8613e-01,
          4.3231e-02, -3.8118e-01,  4.4963e-02, -6.1407e-01, -2.4469e-01,
         -6.8058e-02,  3.5061e-02, -6.1081e-01, -3.4325e-01,  7.0763e-01],
        [-3.7448e-02,  6.8192e-02,  5.0094e+00,  3.1936e-02, -9.1801e-01,
         -9.2670e-02,  1.3304e-01,  2.8113e-02,  1.8417e-01, -8.7350e-02,
         -8.1643e-02, -3.9441e-01, -2.6840e-01,  7.7351e-02,  3.4614e-01,
   

In [95]:
def custom_lr_lambda(step):
    num_step_threshold = 100

    if step < num_step_threshold:
        return step / num_step_threshold
    if step == num_step_threshold:
        print("here")
    return 0.99999 ** (step - num_step_threshold)

start_time = time.time()
model = TabularDenseNet(num_features, num_classes, 1, 2).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001 * 0.1)
scheduler = LambdaLR(optimizer, lr_lambda=custom_lr_lambda)

criterion = CustomLoss(nn.CrossEntropyLoss(), 0.0, 0.0, 0.0, 0.0, 0.0)
evaluate_model(model, custom_train_loader, criterion, optimizer, 10000, scheduler, batch_size, num_features, early_stopping_patience=10000)

print(f"Execution time: {(time.time() - start_time):.6f} seconds")

here
Learning Rate: 9.998700077997147e-05
Epoch 1, Training Loss: 0.3454318817470744, Validation Loss: 0.26897857872054437
Epoch 1, Training Loss: 0.26728401439349014, Validation Loss: 0.26897857872054437
Training Accuracy: 0.8973240929076891, Training F1 Score: 0.8914668805126142
Validation Accuracy: 0.898059848733969, Validation F1 Score: 0.8924544424941138

Learning Rate: 9.886350177801952e-05
Epoch 11, Training Loss: 0.21274061212868917, Validation Loss: 0.22232411616101777
Epoch 11, Training Loss: 0.21116409075468082, Validation Loss: 0.22232411616101777
Training Accuracy: 0.9329378950830055, Training F1 Score: 0.9291547017317171
Validation Accuracy: 0.9285322810479009, Validation F1 Score: 0.9245266797676718

Learning Rate: 9.775262691718132e-05
Epoch 21, Training Loss: 0.20356943747247294, Validation Loss: 0.215115248020132
Epoch 21, Training Loss: 0.20241574494068862, Validation Loss: 0.215115248020132
Training Accuracy: 0.9360802893925922, Training F1 Score: 0.9325798659201071

KeyboardInterrupt: 

In [94]:
def custom_lr_lambda(step):
    num_step_threshold = 100

    if step < num_step_threshold:
        return step / num_step_threshold
    if step == num_step_threshold:
        print("here")
    return 0.99995 ** (step - num_step_threshold)

start_time = time.time()
model = TabularDenseNet(num_features, num_classes, 1, 2).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001 * 0.1)
scheduler = LambdaLR(optimizer, lr_lambda=custom_lr_lambda)

criterion = CustomLoss(nn.CrossEntropyLoss(), 0.0, 0.0, 0.0, 0.0, 0.0)
evaluate_model(model, custom_train_loader, criterion, optimizer, 10000, scheduler, batch_size, num_features, early_stopping_patience=10000)

print(f"Execution time: {(time.time() - start_time):.6f} seconds")

here
Learning Rate: 9.993501949642547e-05
Epoch 1, Training Loss: 0.34543187213408943, Validation Loss: 0.2689781817711198
Epoch 1, Training Loss: 0.26728370172882226, Validation Loss: 0.2689781817711198
Training Accuracy: 0.8973119130847838, Training F1 Score: 0.8914560382205152
Validation Accuracy: 0.898059848733969, Validation F1 Score: 0.8924544424941138

Learning Rate: 9.444510414519057e-05
Epoch 11, Training Loss: 0.21289476665799534, Validation Loss: 0.2224465758065755
Epoch 11, Training Loss: 0.21131804840854507, Validation Loss: 0.2224465758065755
Training Accuracy: 0.932889175791384, Training F1 Score: 0.9291719103274619
Validation Accuracy: 0.9287515071796558, Validation F1 Score: 0.9248588465848174

Learning Rate: 8.925677647278535e-05
Epoch 21, Training Loss: 0.2036504547566547, Validation Loss: 0.21515626204120972
Epoch 21, Training Loss: 0.20259356287623168, Validation Loss: 0.21515626204120972
Training Accuracy: 0.9355687368305665, Training F1 Score: 0.9319644311150924


KeyboardInterrupt: 

In [87]:
def custom_lr_lambda(step):
    num_step_threshold = 100

    if step < num_step_threshold:
        return step / num_step_threshold
    if step == num_step_threshold:
        print("here")
    return 0.99995 ** (step - num_step_threshold)

start_time = time.time()
model = TabularDenseNet(num_features, num_classes, 1, 1).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001 * 1.0)
scheduler = LambdaLR(optimizer, lr_lambda=custom_lr_lambda)

criterion = CustomLoss(nn.CrossEntropyLoss(), 0.0, 0.0, 0.0, 0.0, 0.0)
evaluate_model(model, custom_train_loader, criterion, optimizer, 1000, scheduler, batch_size, num_features, early_stopping_patience=10000)

print(f"Execution time: {(time.time() - start_time):.6f} seconds")

here
Learning Rate: 0.0009993501949642546
Epoch 1, Training Loss: 0.32546490229846003, Validation Loss: 0.2659295236745529
Epoch 1, Training Loss: 0.26357052380919055, Validation Loss: 0.2659295236745529
Training Accuracy: 0.902354359767609, Training F1 Score: 0.8975328304177069
Validation Accuracy: 0.9020059191055574, Validation F1 Score: 0.8970952198205078

Learning Rate: 0.0009444510414519056
Epoch 11, Training Loss: 0.20728066678364604, Validation Loss: 0.21831160964417534
Epoch 11, Training Loss: 0.20480267187329348, Validation Loss: 0.21831160964417534
Training Accuracy: 0.9343994738316505, Training F1 Score: 0.9313326827915461
Validation Accuracy: 0.9296284117066754, Validation F1 Score: 0.9262941965161179

Learning Rate: 0.0008925677647278533
Epoch 21, Training Loss: 0.20160902751758486, Validation Loss: 0.21422199292734606
Epoch 21, Training Loss: 0.19994564647568017, Validation Loss: 0.21422199292734606
Training Accuracy: 0.9369694164646846, Training F1 Score: 0.9338666993206

In [ ]:
def custom_lr_lambda(step):
    num_step_threshold = 100

    if step < num_step_threshold:
        return step / num_step_threshold
    if step == num_step_threshold:
        print("here")
    return 0.99995 ** (step - num_step_threshold)

start_time = time.time()
model = TabularDenseNet(num_features, num_classes, 1, 2).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001 * 0.1)
scheduler = LambdaLR(optimizer, lr_lambda=custom_lr_lambda)

criterion = CustomLoss(nn.CrossEntropyLoss(), 0.0, 0.0, 0.0, 0.0, 0.0)
evaluate_model(model, custom_train_loader, criterion, optimizer, 100, scheduler, batch_size, num_features, early_stopping_patience=10000)

print(f"Execution time: {(time.time() - start_time):.6f} seconds")

In [54]:
def custom_lr_lambda(step):
    num_step_threshold = 100

    if step < num_step_threshold:
        return step / num_step_threshold
    if step == num_step_threshold:
        print("here")
    return 0.99995 ** (step - num_step_threshold)

start_time = time.time()
model = TabularDenseNet(num_features, num_classes, 1, 2).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001 * 0.1)
scheduler = LambdaLR(optimizer, lr_lambda=custom_lr_lambda)

criterion = CustomLoss(nn.CrossEntropyLoss(), 0.0, 0.0, 0.0, 0.0, 0.0)
evaluate_model(model, custom_train_loader, criterion, optimizer, 100, scheduler, batch_size, num_features, early_stopping_patience=10000)

print(f"Execution time: {(time.time() - start_time):.6f} seconds")

here
Learning Rate: 9.988006897470666e-05
Epoch 1, Training Loss: 0.32685440889212714, Validation Loss: 0.257282424665098
Epoch 1, Training Loss: 0.2592016404864994, Validation Loss: 0.257282424665098
Training Accuracy: 0.9026828917210147, Training F1 Score: 0.8963617331047248
Validation Accuracy: 0.9036144578313253, Validation F1 Score: 0.8958853391757421

Learning Rate: 9.387542204897366e-05
Epoch 11, Training Loss: 0.21140293809030944, Validation Loss: 0.22028404990005077
Epoch 11, Training Loss: 0.20871767396398924, Validation Loss: 0.22028404990005077
Training Accuracy: 0.932457121344657, Training F1 Score: 0.9286888325130336
Validation Accuracy: 0.932092004381161, Validation F1 Score: 0.9274332304769843

Learning Rate: 8.823176591021984e-05
Epoch 21, Training Loss: 0.2023875090350066, Validation Loss: 0.21680489771663816
Epoch 21, Training Loss: 0.19974285401390293, Validation Loss: 0.21680489771663816
Training Accuracy: 0.9368086543465504, Training F1 Score: 0.9333338854847838
V

In [52]:
def custom_lr_lambda(step):
    num_step_threshold = 100

    if step < num_step_threshold:
        return step / num_step_threshold
    if step == num_step_threshold:
        print("here")
    return 0.99995 ** (step - num_step_threshold)

start_time = time.time()
model = TabularDenseNet(num_features, num_classes, 1, 2).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001 * 0.1)
scheduler = LambdaLR(optimizer, lr_lambda=custom_lr_lambda)

criterion = CustomLoss(nn.CrossEntropyLoss(), 0.0, 0.0, 0.0, 0.0, 0.0)
evaluate_model(model, custom_train_loader, criterion, optimizer, 100, scheduler, batch_size, num_features, early_stopping_patience=10000)

print(f"Execution time: {(time.time() - start_time):.6f} seconds")

here
Learning Rate: 9.988006897470666e-05
Epoch 1, Training Loss: 0.32683102810382714, Validation Loss: 0.25724726628108197
Epoch 1, Training Loss: 0.25916707453942556, Validation Loss: 0.25724726628108197
Training Accuracy: 0.9026939643240729, Training F1 Score: 0.8963753450799378
Validation Accuracy: 0.9036144578313253, Validation F1 Score: 0.8958853391757421

Learning Rate: 9.387542204897366e-05
Epoch 11, Training Loss: 0.21122512396874166, Validation Loss: 0.22014264380487922
Epoch 11, Training Loss: 0.2085638572099276, Validation Loss: 0.22014264380487922
Training Accuracy: 0.9326896460088803, Training F1 Score: 0.9289523711871929
Validation Accuracy: 0.932092004381161, Validation F1 Score: 0.9274332304769843

Learning Rate: 8.823176591021984e-05
Epoch 21, Training Loss: 0.20216124986696216, Validation Loss: 0.21774567759964056
Epoch 21, Training Loss: 0.1997491588014111, Validation Loss: 0.21774567759964056
Training Accuracy: 0.9367754365373756, Training F1 Score: 0.9332525922781

In [50]:
def custom_lr_lambda(step):
    num_step_threshold = 100

    if step < num_step_threshold:
        return step / num_step_threshold
    if step == num_step_threshold:
        print("here")
    return 0.99995 ** (step - num_step_threshold)

start_time = time.time()
model = TabularDenseNet(num_features, num_classes, 1, 2).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001 * 0.1)
scheduler = LambdaLR(optimizer, lr_lambda=custom_lr_lambda)

criterion = CustomLoss(nn.CrossEntropyLoss(), 0.0, 0.0, 0.0, 0.0, 0.0)
evaluate_model(model, custom_train_loader, criterion, optimizer, 100, scheduler, batch_size, num_features, early_stopping_patience=10000)

print(f"Execution time: {(time.time() - start_time):.6f} seconds")

here
Learning Rate: 9.988006897470666e-05
Epoch 1, Training Loss: 0.3291932054214368, Validation Loss: 0.2653030287984428
Epoch 1, Training Loss: 0.26775688525463903, Validation Loss: 0.2653030287984428
Training Accuracy: 0.8969251381307232, Training F1 Score: 0.8884591794766488
Validation Accuracy: 0.8893756845564075, Validation F1 Score: 0.8798373129465867

Learning Rate: 9.387542204897366e-05
Epoch 11, Training Loss: 0.21218193280767222, Validation Loss: 0.22185845713887
Epoch 11, Training Loss: 0.20957746431130028, Validation Loss: 0.22185845713887
Training Accuracy: 0.932457121344657, Training F1 Score: 0.9291982888669961
Validation Accuracy: 0.9299014238773274, Validation F1 Score: 0.925861893260901

Learning Rate: 8.823176591021984e-05
Epoch 21, Training Loss: 0.20438041422660602, Validation Loss: 0.2158162516642766
Epoch 21, Training Loss: 0.20188207492690513, Validation Loss: 0.2158162516642766
Training Accuracy: 0.9356460310254338, Training F1 Score: 0.9326249312383817
Valida

In [35]:
def custom_lr_lambda(step):
    num_step_threshold = 100

    if step < num_step_threshold:
        return step / num_step_threshold
    if step == num_step_threshold:
        print("here")
    return 0.99995 ** (step - num_step_threshold)

start_time = time.time()
model = TabularDenseNet(num_features, num_classes, 1, 2).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001 * 1.0)
scheduler = LambdaLR(optimizer, lr_lambda=custom_lr_lambda)

criterion = CustomLoss(nn.CrossEntropyLoss(), 0.0, 0.0, 0.0, 0.0, 0.0)
evaluate_model(model, custom_train_loader, criterion, optimizer, 1000, scheduler, batch_size, num_features, early_stopping_patience=10000)

print(f"Execution time: {(time.time() - start_time):.6f} seconds")

here
Learning Rate: 0.0009988006897470665
Epoch 1, Training Loss: 5.350201307207334, Validation Loss: 0.5957513029186253
Epoch 1, Training Loss: 0.6417104434654475, Validation Loss: 0.5957513029186253
Training Accuracy: 0.8694650825462558, Training F1 Score: 0.8710187293421613
Validation Accuracy: 0.8608981380065718, Validation F1 Score: 0.8624450227725788

Learning Rate: 0.0009387542204897365
Epoch 11, Training Loss: 0.2374368070791836, Validation Loss: 0.23177352914870242
Epoch 11, Training Loss: 0.22682396977971253, Validation Loss: 0.23177352914870242
Training Accuracy: 0.926378262265676, Training F1 Score: 0.9221026871069838
Validation Accuracy: 0.91894852135816, Validation F1 Score: 0.9130806779673695

Learning Rate: 0.0008823176591021983
Epoch 21, Training Loss: 0.22958999733777136, Validation Loss: 0.22737190346075437
Epoch 21, Training Loss: 0.21862793333916145, Validation Loss: 0.22737190346075437
Training Accuracy: 0.9295450267403363, Training F1 Score: 0.9257459259884844
Va

KeyboardInterrupt: 

In [ ]:
def custom_lr_lambda(step):
    num_step_threshold = 100

    if step < num_step_threshold:
        return step / num_step_threshold
    if step == num_step_threshold:
        print("here")
    return 0.99995 ** (step - num_step_threshold)

start_time = time.time()
model = TabularDenseNet(num_features, num_classes, 1, 8).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001 * 0.01)
scheduler = LambdaLR(optimizer, lr_lambda=custom_lr_lambda)

criterion = CustomLoss(nn.CrossEntropyLoss(), 0.0, 0.0, 0.0, 0.0, 0.0)
evaluate_model(model, custom_train_loader, criterion, optimizer, 1000, scheduler, batch_size, num_features, early_stopping_patience=10000)

print(f"Execution time: {(time.time() - start_time):.6f} seconds")

In [ ]:
def custom_lr_lambda(step):
    num_step_threshold = 100

    if step < num_step_threshold:
        return step / num_step_threshold
    if step == num_step_threshold:
        print("here")
    return 0.99995 ** (step - num_step_threshold)

start_time = time.time()
model = TabularDenseNet(num_features, num_classes, 1, 10).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001 * 0.01)
scheduler = LambdaLR(optimizer, lr_lambda=custom_lr_lambda)

criterion = CustomLoss(nn.CrossEntropyLoss(), 0.0, 0.0, 0.0, 0.0, 0.0)
evaluate_model(model, custom_train_loader, criterion, optimizer, 1000, scheduler, batch_size, num_features, early_stopping_patience=10000)

print(f"Execution time: {(time.time() - start_time):.6f} seconds")

In [51]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total number of trainable parameters: {total_params}")

Total number of trainable parameters: 2051


In [52]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters (trainable + non-trainable): {total_params}")

Total number of parameters (trainable + non-trainable): 2051


In [53]:
data = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv')

data = data.drop(["id"], axis=1)
data['source'] = 0

data['person_emp_length_missing'] = data['person_emp_length'].isna().astype(int)
data['loan_int_rate_missing'] = data['loan_int_rate'].isna().astype(int)

data['person_emp_length'] = data['person_emp_length'].fillna(median_emp_length)
data['loan_int_rate'] = data['loan_int_rate'].fillna(median_int_rate)

# grade_mapping = {'A': 7, 'B': 6, 'C': 5, 'D': 4, 'E': 3, 'F': 2, 'G': 1}
# data['loan_grade'] = data['loan_grade'].map(grade_mapping)

# purpose_mapping = {
#     'DEBTCONSOLIDATION': 1,
#     'HOMEIMPROVEMENT': 2,
#     'MEDICAL': 3,
#     'PERSONAL': 4,
#     'EDUCATION': 5,
#     'VENTURE': 6
# }
# data['loan_intent'] = data['loan_intent'].map(purpose_mapping)

# home_ownership_mapping = {
#     'OWN': 1,
#     'MORTGAGE': 2,
#     'OTHER': 3,
#     'RENT': 4
# }
# data['person_home_ownership'] = data['person_home_ownership'].map(home_ownership_mapping)

X = data.drop([], axis=1)
X = pd.get_dummies(X, drop_first=True)

column_to_log = [
    'person_age',
    'person_income',
]

column_to_sqrt = [
    'person_emp_length',
    'loan_percent_income',
]

for col in column_to_log:
    if (X[col] <= 0).any():
        print(f"Column '{col}' contains non-positive values. Adding 1 to avoid log of non-positive numbers.")
        X[col] = np.log(X[col] + 1)
    else:
        X[col] = np.log(X[col])

for col in column_to_sqrt:
    if (X[col] < 0).any():
        print(f"Column '{col}' contains negative values. Setting negative values to NaN before applying sqrt.")
        X[col] = np.sqrt(X[col].clip(lower=0))
    else:
        X[col] = np.sqrt(X[col])

print(data.isnull().sum())
print(X.columns)
print(X.columns.get_loc('source'))

person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
source                        0
person_emp_length_missing     0
loan_int_rate_missing         0
dtype: int64
Index(['person_age', 'person_income', 'person_emp_length', 'loan_amnt',
       'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length',
       'source', 'person_emp_length_missing', 'loan_int_rate_missing',
       'person_home_ownership_OTHER', 'person_home_ownership_OWN',
       'person_home_ownership_RENT', 'loan_intent_EDUCATION',
       'loan_intent_HOMEIMPROVEMENT', 'loan_intent_MEDICAL',
       'loan_intent_PERSONAL', 'loan_intent_VENTURE', 'loan_grade_B',
       'loan_grade_C', 'loan_grade_D', 'loan_grade_E', 'loa

In [54]:
print(X)

       person_age  person_income  person_emp_length  loan_amnt  loan_int_rate  \
0        3.135494      11.141862           1.732051      25000          15.76   
1        3.258097      11.472103           2.449490      10000          12.68   
2        3.258097      10.308953           2.236068       4000          17.19   
3        3.496508      10.819778           2.000000       7000           8.90   
4        3.258097      11.532728           2.828427      15000          16.32   
...           ...            ...                ...        ...            ...   
39093    3.091042      10.348173           1.414214       3000          10.37   
39094    3.091042      10.778956           2.449490       7000           6.03   
39095    3.931826      11.002100           0.000000      15000           7.51   
39096    3.091042      10.491274           2.000000      14000          15.62   
39097    3.433987      10.714418           2.449490      19450           9.91   

       loan_percent_income 

In [55]:
print(X.shape)
X_scaled_test = x_scaler.transform(X)
print(X_scaled_test.shape)
print(X_scaled_test)

(39098, 25)
(39098, 25)
[[-0.83835155  0.38266958 -0.18616973 ... 11.58407836 -0.04436441
  -0.46268575]
 [-0.21875715  0.96598481  0.50889981 ... -0.08632538 -0.04436441
   2.16129414]
 [-0.21875715 -1.08852135  0.30213243 ... -0.08632538 -0.04436441
   2.16129414]
 ...
 [ 3.18606232  0.13580411 -1.86421602 ... -0.08632538 -0.04436441
  -0.46268575]
 [-1.06299707 -0.76648163  0.07342494 ... -0.08632538 -0.04436441
   2.16129414]
 [ 0.67014014 -0.37233687  0.50889981 ... -0.08632538 -0.04436441
  -0.46268575]]


In [56]:
X_scaled_test_tensor = torch.tensor(X_scaled_test).float().to(device)
outputs = model(X_scaled_test_tensor)
print(outputs)

tensor([[-3.5235,  3.5235],
        [ 1.6366, -1.6366],
        [ 0.4044, -0.4044],
        ...,
        [ 2.3884, -2.3884],
        [ 0.6305, -0.6305],
        [-2.2837,  2.2837]], grad_fn=<StackBackward0>)


In [57]:
probabilities = F.softmax(outputs, dim=1)
print(probabilities)

tensor([[8.6921e-04, 9.9913e-01],
        [9.6350e-01, 3.6503e-02],
        [6.9185e-01, 3.0815e-01],
        ...,
        [9.9165e-01, 8.3532e-03],
        [7.7921e-01, 2.2079e-01],
        [1.0278e-02, 9.8972e-01]], grad_fn=<SoftmaxBackward0>)


In [58]:
positive_class_probs = probabilities[:, 1]
print(positive_class_probs)

tensor([0.9991, 0.0365, 0.3081,  ..., 0.0084, 0.2208, 0.9897],
       grad_fn=<SelectBackward0>)


In [59]:
import pandas as pd

test_df = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv')
ids = test_df['id']

positive_class_probs = positive_class_probs.cpu().detach().numpy()

submission_df = pd.DataFrame({
    'id': ids,
    'loan_status': positive_class_probs
})

submission_df.to_csv('submission.csv', index=False)
print("Submission file created successfully.")

Submission file created successfully.
